In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

from datasets import load_dataset, DatasetDict, load_metric, load_from_disk, Dataset
from transformers import (AutoModelForCausalLM, AutoTokenizer, AutoModelForSeq2SeqLM,
                          Seq2SeqTrainer, Seq2SeqTrainingArguments, default_data_collator, DataCollatorForSeq2Seq)
from transformers import T5Tokenizer, T5ForConditionalGeneration, BartTokenizer
from transformers import TFAutoModelForQuestionAnswering, BartTokenizer
import nltk
nltk.download('punkt')
import string
import numpy as np
import pandas as pd
from evaluate import load
import torch
import evaluate

[nltk_data] Downloading package punkt to /home/ubuntu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"

## Load Data and remove external QA pairs

In [3]:
df = pd.read_excel('/home/ubuntu/QuestionAnswering/WOT_internal_refined_4.xlsx')

df['comment'] = df['comment'].fillna('')

# remove external Question pairs
df['not external'] = df['comment'].apply(lambda x: 'external' not in x.split(' + '))
df = df[df['not external']]

df = df[['domain', 'data_split', 'question', 'history', 'Context', 'answer_extr', 'answer_start', 'number_answer_elements', 'comment']].rename(columns={'Context': 'context', 'answer_extr': 'answers'})

df.to_csv('/home/ubuntu/QuestionAnswering/Data/temp.csv')
df = pd.read_csv('/home/ubuntu/QuestionAnswering/Data/temp.csv').rename(columns={'Unnamed: 0': 'id'})
df['id'] = df['id'].astype(str)

df['answers'] = df['answers'].apply(lambda x: x[2:-2].split("\', \'"))
df['answer_start'] = df['answer_start'].apply(lambda x: x[1:-1].split(", "))
df['answer_start'] = df['answer_start'].apply(lambda x: [-1] if x[0]=='' else x)
df['answer_start'] = df['answer_start'].apply(lambda x: [int(el) for el in x])
df['answers'] = df.apply(lambda row: {'text': row['answers'], 'answer_start': row['answer_start']} , axis=1)

## Define Input to feed to the model

In [4]:
df['input'] = df.apply(lambda row: f"{row['question']}SPLITInstructions: {row['context']}\nHistory: {row['history']}\n", axis=1)

## One span answers only

In [5]:
df_1_span = df[df['number_answer_elements'] == 1]
df_1_span['output'] = df['answers'].apply(lambda x: x['text'][0])

ds_1_span_train = Dataset.from_pandas(df_1_span[df_1_span['data_split']=='train'].reset_index())
ds_1_span_test = Dataset.from_pandas(df_1_span[df_1_span['data_split']=='test'].reset_index())
ds_1_span_val = Dataset.from_pandas(df_1_span[df_1_span['data_split']=='validation'].reset_index())

ds_1_span = DatasetDict({'train': ds_1_span_train,
                         'test': ds_1_span_test,
                         'validation': ds_1_span_val})


ds_1_span_squad =  Dataset.from_pandas(df_1_span)
print(ds_1_span)

DatasetDict({
    train: Dataset({
        features: ['index', 'id', 'domain', 'data_split', 'question', 'history', 'context', 'answers', 'answer_start', 'number_answer_elements', 'comment', 'input', 'output'],
        num_rows: 661
    })
    test: Dataset({
        features: ['index', 'id', 'domain', 'data_split', 'question', 'history', 'context', 'answers', 'answer_start', 'number_answer_elements', 'comment', 'input', 'output'],
        num_rows: 64
    })
    validation: Dataset({
        features: ['index', 'id', 'domain', 'data_split', 'question', 'history', 'context', 'answers', 'answer_start', 'number_answer_elements', 'comment', 'input', 'output'],
        num_rows: 72
    })
})


/tmp/ipykernel_23850/3182053230.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1_span['output'] = df['answers'].apply(lambda x: x['text'][0])


In [6]:
model_name = "allenai/unifiedqa-t5-large"
#model_name = 't5-base'
tokenizer = T5Tokenizer.from_pretrained(model_name)

# BART
# model_name = "facebook/bart-base"
# tokenizer = BartTokenizer.from_pretrained(model_name)

model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
model_dir = f"Models/extractive_WoT/{model_name}"

In [8]:
prefix = "question: "
max_input_length = 512
max_target_length = 50

def clean_text(text):
    question = nltk.sent_tokenize(text.split('SPLIT')[0])
    sentences = nltk.sent_tokenize(text.split('SPLIT')[1])
    text_cleaned = "\n ".join([" ".join(question).lower(), " ".join(sentences).lower()])
    return text_cleaned

def preprocess_data(examples):
    texts_cleaned = [clean_text(text) for text in examples["input"]]
    inputs = [prefix + text for text in texts_cleaned]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True).to(device)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["output"], max_length=max_target_length, truncation=True).to(device)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [8]:
tokenized_train = ds_1_span['train'].map(preprocess_data, batched=True)
tokenized_dev = ds_1_span['validation'].map(preprocess_data, batched=True)

Map:   0%|          | 0/661 [00:00<?, ? examples/s]

AttributeError: 'list' object has no attribute 'to'

## Untuned

In [12]:
# pad texts to the same length
def preprocess_test(examples):
    texts_cleaned = [clean_text(text) for text in examples["input"]]
    inputs = [prefix + text for text in texts_cleaned]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True,
                            padding="max_length")
    return model_inputs

In [ ]:
test_tokenized_dataset = ds_1_span['test'].map(preprocess_test, batched=True)

# prepare dataloader
test_tokenized_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask'])
dataloader = torch.utils.data.DataLoader(test_tokenized_dataset, batch_size=32)

# generate text for each batch
all_predictions = []
for i,batch in enumerate(dataloader):
    predictions = model.generate(**batch)
    all_predictions.append(predictions)

# flatten predictions
all_predictions_flattened = [pred for preds in all_predictions for pred in preds]

# tokenize and pad titles
all_titles = tokenizer(test_tokenized_dataset["output"], max_length=max_target_length,
                       truncation=True, padding="max_length")["input_ids"]

# compute metrics
decoded_preds = tokenizer.batch_decode(all_predictions_flattened, skip_special_tokens=True)

labels = np.where(all_titles != -100, all_titles, tokenizer.pad_token_id)
decoded_labels_raw = tokenizer.batch_decode(labels, skip_special_tokens=True)

exact_match_metric = evaluate.load("exact_match")
exact_match_metric.compute(predictions=decoded_preds, references=decoded_labels_raw)

Map:   0%|          | 0/64 [00:00<?, ? examples/s]

/usr/local/lib/python3.8/dist-packages/transformers/generation/utils.py:1273: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 20 (`generation_config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


{'exact_match': 0.046875}

## Fine tune

In [7]:
per_device_eval_batch_size = 64     #64
per_device_train_batch_size = 16     #16
gradient_accumulation_steps = 4     # per_device_train_batch_size * gradient_accumulation_steps=64 change here to be 64

args = Seq2SeqTrainingArguments(
    model_dir,
    evaluation_strategy="steps",
    eval_steps=100,
    logging_strategy="steps",
    logging_steps=100,
    save_strategy="steps",
    save_steps=100,
    learning_rate=4e-5,
    per_device_train_batch_size=per_device_train_batch_size,
    per_device_eval_batch_size=per_device_eval_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    weight_decay=0.01,
    predict_with_generate=True,
    fp16=True,
    load_best_model_at_end=True,
    metric_for_best_model="rouge1",
    report_to="tensorboard",
    max_steps=5000,
    optim='adafactor'
)

In [14]:
data_collator = DataCollatorForSeq2Seq(tokenizer)

In [8]:
metric = load_metric('rouge')
exact_match_metric = load("exact_match")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels_raw = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    
    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip()))
                      for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) 
                      for label in decoded_labels_raw]
    decoded_labels_list = [["\n".join(nltk.sent_tokenize(label.strip()))] 
                      for label in decoded_labels_raw]

    # Compute ROUGE scores
    result = metric.compute(predictions=decoded_preds, references=decoded_labels,
                            use_stemmer=True)
    
    # Extract ROUGE f1 scores
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}

    #result['bleu'] = bleu.compute(predictions=decoded_preds, references=decoded_labels_list)['bleu']
    result['exact_match'] = exact_match_metric.compute(predictions=decoded_preds, references=decoded_labels)['exact_match']
    #result_bert_score = bert_score.compute(predictions=decoded_preds, references=decoded_labels_list, lang="en")['f1']
    #result['bert_score (avg. F1)'] = sum(result_bert_score) / len(result_bert_score)
    
    # Add mean generated length to metrics
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id)
                      for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    with open('/home/ubuntu/QuestionAnswering/logging', 'a') as log:
        log.write("\n" + str({k: round(v, 4) for k, v in result.items()}))
        
    return {k: round(v, 4) for k, v in result.items()}

In [16]:
model_checkpoint = model_name

def model_init():
    return AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

trainer = Seq2SeqTrainer(
    model_init=model_init,
    args=args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_dev,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--allenai--unifiedqa-t5-base/snapshots/e72d6077d80b8a2e20e602c6ecf396e04f920746/config.json
Model config T5Config {
  "_name_or_path": "allenai/unifiedqa-t5-base",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "d_ff": 3072,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dense_act_fn": "relu",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": false,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 12,
  "num_heads": 12,
  "num_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty": 2.0,
      "max_length": 200,
      "min_length": 30

In [ ]:
trainer.train()

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--allenai--unifiedqa-t5-base/snapshots/e72d6077d80b8a2e20e602c6ecf396e04f920746/config.json
Model config T5Config {
  "_name_or_path": "allenai/unifiedqa-t5-base",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "d_ff": 3072,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dense_act_fn": "relu",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": false,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 12,
  "num_heads": 12,
  "num_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty": 2.0,
      "max_length": 200,
      "min_length": 30

Step,Training Loss,Validation Loss


OutOfMemoryError: ignored

### Evaluate Fine tuned

In [ ]:
checkpoint = "checkpoint-100"
model_dir = f"{model_dir}/{checkpoint}"

tokenizer = AutoTokenizer.from_pretrained(model_dir)
model = AutoModelForSeq2SeqLM.from_pretrained(model_dir)

max_input_length = 512

loading file vocab.json
loading file merges.txt
loading file tokenizer.json
loading file added_tokens.json
loading file special_tokens_map.json
loading file tokenizer_config.json
loading configuration file drive/MyDrive/Models/extractive_WoT/facebook/bart-base/checkpoint-100/config.json
Model config BartConfig {
  "_name_or_path": "drive/MyDrive/Models/extractive_WoT/facebook/bart-base/checkpoint-100",
  "activation_dropout": 0.1,
  "activation_function": "gelu",
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "architectures": [
    "BartForConditionalGeneration"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 0,
  "classif_dropout": 0.1,
  "classifier_dropout": 0.0,
  "d_model": 768,
  "decoder_attention_heads": 12,
  "decoder_ffn_dim": 3072,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 6,
  "decoder_start_token_id": 2,
  "dropout": 0.1,
  "early_stopping": true,
  "encoder_attention_heads": 12,
  "encoder_ffn_dim": 3072,
  "encoder_layerdrop": 0.0,
  "encoder_lay

In [ ]:
text = ds_1_span['test']['input'][0]

inputs = ["question: " + text]

inputs = tokenizer(inputs, max_length=max_input_length, truncation=True, return_tensors="pt")
output = model.generate(**inputs, num_beams=8, do_sample=True, min_length=10, max_length=64)
decoded_output = tokenizer.batch_decode(output, skip_special_tokens=True)[0]
predicted_answer = nltk.sent_tokenize(decoded_output.strip())[0]

print(text)
print(f'Answer: {ds_1_span["test"]["output"][0]}')
print(predicted_answer)

Generate config GenerationConfig {
  "bos_token_id": 0,
  "decoder_start_token_id": 2,
  "early_stopping": true,
  "eos_token_id": 2,
  "forced_bos_token_id": 0,
  "forced_eos_token_id": 2,
  "no_repeat_ngram_size": 3,
  "num_beams": 4,
  "pad_token_id": 1,
  "transformers_version": "4.26.1"
}



How long should they be blended after combining? SPLITInstructions: Title:
chocolate-peanut banana shake

|Description:
 this refreshing milkshake makes the most of our favorite trio of sweet flavors: chocolate, peanut butter and banana.

|Ingredients:
2 cups chocolate soymilk
2 tablespoons creamy peanut butter
1 banana, broken into chunks
4 ice cubes
pinch grated nutmeg or ground cinnamon, (optional)

|Steps:
combine soymilk, peanut butter, banana and ice in a blender and blend until smooth.
pour into 2 tall glasses and sprinkle with nutmeg.


History: student: Previous step to use to make and cook the milk shakes and chocolate-peanut banana shakes | teacher: Could you please clarify your question?  Have you assembled the ingredients called for?  I've shared a list for your reference.  | student: What should I do with the ingredients? | teacher: The first step is to combine your soy milk, peanut butter, banana, and ice into your blender. 

Answer: blend until smooth
blend until smooth

In [ ]:
import torch

# pad texts to the same length
def preprocess_test(examples):
    texts_cleaned = [clean_text(text) for text in examples["input"]]
    inputs = [prefix + text for text in texts_cleaned]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True,
                            padding="max_length")
    return model_inputs

test_tokenized_dataset = ds_1_span['test'].map(preprocess_test, batched=True)

# prepare dataloader
test_tokenized_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask'])
dataloader = torch.utils.data.DataLoader(test_tokenized_dataset, batch_size=32)

# generate text for each batch
all_predictions = []
for i,batch in enumerate(dataloader):
  predictions = model.generate(**batch)
  all_predictions.append(predictions)

# flatten predictions
all_predictions_flattened = [pred for preds in all_predictions for pred in preds]

# tokenize and pad titles
all_titles = tokenizer(test_tokenized_dataset["output"], max_length=max_target_length,
                       truncation=True, padding="max_length")["input_ids"]

# compute metrics
predictions_labels = [all_predictions_flattened, all_titles]
compute_metrics(predictions_labels)

Map:   0%|          | 0/64 [00:00<?, ? examples/s]

Generate config GenerationConfig {
  "bos_token_id": 0,
  "decoder_start_token_id": 2,
  "early_stopping": true,
  "eos_token_id": 2,
  "forced_bos_token_id": 0,
  "forced_eos_token_id": 2,
  "no_repeat_ngram_size": 3,
  "num_beams": 4,
  "pad_token_id": 1,
  "transformers_version": "4.26.1"
}

/usr/local/lib/python3.8/dist-packages/transformers/generation/utils.py:1273: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 20 (`generation_config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
Generate config GenerationConfig {
  "bos_token_id": 0,
  "decoder_start_token_id": 2,
  "early_stopping": true,
  "eos_token_id": 2,
  "forced_bos_token_id": 0,
  "forced_eos_token_id": 2,
  "no_repeat_ngram_size": 3,
  "num_beams": 4,
  "pad_token_id": 1,
  "

{'rouge1': 30.1791,
 'rouge2': 17.7827,
 'rougeL': 28.4846,
 'rougeLsum': 28.7201,
 'exact_match': 0.0938,
 'gen_len': 13.0938}

In [ ]:
def input_length(input):
    text_cleaned = clean_text(input)
    input = prefix + text_cleaned
    model_input = tokenizer(input)
    return len(model_input['input_ids'])

def output_length(output):
    return len(tokenizer(output)["input_ids"])

df_test = df_1_span[df_1_span['data_split']=='test']
decoded_preds = tokenizer.batch_decode(all_predictions_flattened, skip_special_tokens=True)
df_test['prediction'] = decoded_preds
df_test['exact_match'] = df_test['prediction'] == df_test['output']
df_test['average_input_length'] = df_test['input'].apply(input_length)
df_test['average_output_length'] = df_test['output'].apply(output_length)
df_test['truncation_rate'] = df_test['average_input_length'] > 512

<ipython-input-17-726f1e0d523a>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['prediction'] = decoded_preds
<ipython-input-17-726f1e0d523a>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['exact_match'] = df_test['prediction'] == df_test['output']
Token indices sequence length is longer than the specified maximum sequence length for this model (2249 > 1024). Running this sequence through the model will result in indexing errors
<ipython-input-17-726f1e0d523a>:14: SettingWithCopyWarning

In [ ]:
df_test.groupby('domain').mean().reset_index()[['domain', 'exact_match', 'average_input_length', 'truncation_rate']]

,domain,exact_match,average_input_length,truncation_rate
0,cooking,0.185185,376.592593,0.074074
1,diy,0.027027,1629.378378,1.000000


In [ ]:
df_test['comment'] = df_test['comment'].fillna('easy')
df_test.groupby('comment').mean().reset_index()[['comment', 'exact_match']].merge(df_test.groupby('comment').count().reset_index()[['comment', 'id']].rename(columns={'id':'count'}), on='comment')

<ipython-input-19-24f6d790c610>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['comment'] = df_test['comment'].fillna('easy')


,comment,exact_match,count
0,easy,0.122449,49
1,history,0.000000,2
2,no,0.000000,3
3,not specified,0.000000,2
4,reasoning,0.000000,4
5,yes,0.000000,2
6,yes + reasoning,0.000000,2


In [ ]:
df_test.groupby(['domain','exact_match']).mean().reset_index()[['domain','exact_match', 'average_output_length']].merge(df_test.groupby(['domain','exact_match']).count().reset_index()[['domain','exact_match', 'id']].rename(columns={'id':'count'}), on=['domain','exact_match'])

,domain,exact_match,average_output_length,count
0,cooking,False,15.818182,22
1,cooking,True,6.600000,5
2,diy,False,16.638889,36
3,diy,True,12.000000,1


In [ ]:
df_test.to_excel('/content/temp.xlsx')

## Very Few Shot Learning

In [ ]:
tokenized_train = ds_1_span['test'].map(preprocess_data, batched=True)
tokenized_dev = ds_1_span['validation'].map(preprocess_data, batched=True)

model_dir = f"drive/MyDrive/Models/extractive_WoT/{model_name}/very_few_shot"

Map:   0%|          | 0/64 [00:00<?, ? examples/s]

/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:3581: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/72 [00:00<?, ? examples/s]

In [32]:
args = Seq2SeqTrainingArguments(
    model_dir,
    evaluation_strategy="steps",
    eval_steps=100,
    logging_strategy="steps",
    logging_steps=100,
    save_strategy="steps",
    save_steps=100,
    learning_rate=4e-5,
    per_device_train_batch_size=per_device_train_batch_size,
    per_device_eval_batch_size=per_device_eval_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    weight_decay=0.01,
    predict_with_generate=True,
    fp16=True,
    load_best_model_at_end=True,
    metric_for_best_model="rouge1",
    report_to="tensorboard",
    optim="adafactor",
    max_steps=5000
)


model_checkpoint = model_name

def model_init():
    return AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint).to_cuda()

trainer = Seq2SeqTrainer(
    model_init=model_init,
    args=args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_dev,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()

PyTorch: setting up devices
loading configuration file config.json from cache at /home/ubuntu/.cache/huggingface/hub/models--allenai--unifiedqa-t5-base/snapshots/e72d6077d80b8a2e20e602c6ecf396e04f920746/config.json
Model config T5Config {
  "_name_or_path": "allenai/unifiedqa-t5-base",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "d_ff": 3072,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dense_act_fn": "relu",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": false,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 12,
  "num_heads": 12,
  "num_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty": 2.0,
      "max_l

AttributeError: 'T5ForConditionalGeneration' object has no attribute 'to_cuda'

### Evalute

In [ ]:
checkpoint = "checkpoint-100"
model_dir = f"{model_dir}/{checkpoint}"

tokenizer = AutoTokenizer.from_pretrained(model_dir)
model = AutoModelForSeq2SeqLM.from_pretrained(model_dir)

max_input_length = 512

test_tokenized_dataset = ds_1_span['train'].map(preprocess_test, batched=True)

# prepare dataloader
test_tokenized_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask'])
dataloader = torch.utils.data.DataLoader(test_tokenized_dataset, batch_size=32)

# generate text for each batch
all_predictions = []
for i,batch in enumerate(dataloader):
  predictions = model.generate(**batch)
  all_predictions.append(predictions)

# flatten predictions
all_predictions_flattened = [pred for preds in all_predictions for pred in preds]

# tokenize and pad titles
all_titles = tokenizer(test_tokenized_dataset["output"], max_length=max_target_length,
                       truncation=True, padding="max_length")["input_ids"]

# compute metrics
predictions_labels = [all_predictions_flattened, all_titles]
compute_metrics(predictions_labels)

loading file spiece.model
loading file tokenizer.json
loading file added_tokens.json
loading file special_tokens_map.json
loading file tokenizer_config.json
loading configuration file drive/MyDrive/Models/extractive_WoT/allenai/unifiedqa-t5-base/very_few_shot/checkpoint-100/config.json
Model config T5Config {
  "_name_or_path": "drive/MyDrive/Models/extractive_WoT/allenai/unifiedqa-t5-base/very_few_shot/checkpoint-100",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "d_ff": 3072,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dense_act_fn": "relu",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": false,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 12,
  "num_heads": 12,
  "num_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num

Map:   0%|          | 0/661 [00:00<?, ? examples/s]

Generate config GenerationConfig {
  "decoder_start_token_id": 0,
  "eos_token_id": 1,
  "pad_token_id": 0,
  "transformers_version": "4.26.1"
}

/usr/local/lib/python3.8/dist-packages/transformers/generation/utils.py:1273: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 20 (`generation_config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
Generate config GenerationConfig {
  "decoder_start_token_id": 0,
  "eos_token_id": 1,
  "pad_token_id": 0,
  "transformers_version": "4.26.1"
}

Generate config GenerationConfig {
  "decoder_start_token_id": 0,
  "eos_token_id": 1,
  "pad_token_id": 0,
  "transformers_version": "4.26.1"
}

Generate config GenerationConfig {
  "decoder_start_token_id": 0,
  "eos_token_id": 1,
  "pad_token_id": 0,
  "transfor

{'rouge1': 32.0759,
 'rouge2': 24.0029,
 'rougeL': 31.2158,
 'rougeLsum': 31.2191,
 'exact_match': 0.0998,
 'gen_len': 10.6369}

In [ ]:
def input_length(input):
    text_cleaned = clean_text(input)
    input = prefix + text_cleaned
    model_input = tokenizer(input)
    return len(model_input['input_ids'])

def output_length(output):
    return len(tokenizer(output)["input_ids"])
    
df_test = df_1_span[df_1_span['data_split']=='train']
decoded_preds = tokenizer.batch_decode(all_predictions_flattened, skip_special_tokens=True)
df_test['prediction'] = decoded_preds
df_test['exact_match'] = df_test['prediction'] == df_test['output']
df_test['average_input_length'] = df_test['input'].apply(input_length)
df_test['average_output_length'] = df_test['output'].apply(output_length)
df_test['truncation_rate'] = df_test['average_input_length'] > 512

<ipython-input-26-b4620f2ed983>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['prediction'] = decoded_preds
<ipython-input-26-b4620f2ed983>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['exact_match'] = df_test['prediction'] == df_test['output']
Token indices sequence length is longer than the specified maximum sequence length for this model (550 > 512). Running this sequence through the model will result in indexing errors
<ipython-input-26-b4620f2ed983>:14: SettingWithCopyWarning: 

In [ ]:
df_test.groupby('domain').mean().reset_index()[['domain', 'exact_match', 'average_input_length', 'truncation_rate']]

,domain,exact_match,average_input_length,truncation_rate
0,cooking,0.195205,420.222603,0.157534
1,diy,0.024390,1821.555556,1.000000


In [ ]:
df_test['comment'] = df_test['comment'].fillna('easy')
df_test.groupby('comment').mean().reset_index()[['comment', 'exact_match']].merge(df_test.groupby('comment').count().reset_index()[['comment', 'id']].rename(columns={'id':'count'}), on='comment')

<ipython-input-28-24f6d790c610>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['comment'] = df_test['comment'].fillna('easy')


,comment,exact_match,count
0,context,0.000000,2
1,easy,0.102845,457
2,end,0.000000,3
3,end + history,0.000000,1
4,history,0.066667,30
5,history + end,0.000000,1
6,history + reasoning,0.000000,1
7,no,0.100000,40
8,no + context,0.000000,1
9,no + history,0.250000,4


# Fine Tune on other QA dataset

In [9]:
from datasets import load_dataset

ds_coqa = load_dataset("coqa")
df_coqa = pd.DataFrame(ds_coqa['train'])

source_id = []
source = []
context = []
questions = []
answers = []
history = []
ans_start = []
ans_end = []

prev_utterances = 4


def create_conv(questions, answers):
    out = [''] * 2 * len(questions)
    out[::2] = questions
    out[1::2] = answers
    return out

df_coqa['conversation'] = df_coqa.apply(lambda row: create_conv(row['questions'], row['answers']['input_text']), axis=1)

for i, row in df_coqa.iterrows():
    for j, question in enumerate(row['questions']):
        context.append(row['story'])
        questions.append(question)
        ans_start = row['answers']['answer_start'][j]
        ans_end = row['answers']['answer_end'][j]
        answers.append(row['story'][ans_start:ans_end])
        history.append(row['conversation'][max(0, j*2-4):j*2])
        source_id.append(i)
        source.append(row['source'])
     
df_coqa_wot = pd.DataFrame(data={'source_id': source_id, 'source': source, 'context': context, 'question': questions, 'answer':answers, 'history': history})

df_coqa_wot['input'] = df_coqa_wot.apply(lambda row: f"{row['question']}SPLITContext: {row['context']}\nHistory: {row['history']}\n", axis=1)
df_coqa_wot['output'] = df_coqa_wot['answer']

df_coqa_wot = df_coqa_wot[(df_coqa_wot['output'].apply(lambda x: len(x)) > 0) & (df_coqa_wot['output'].apply(lambda x: len(x.split(' '))) < 20)]
#df_coqa_wot = df_coqa_wot.sample(50000)

Found cached dataset coqa (/home/ubuntu/.cache/huggingface/datasets/coqa/default/1.0.0/1b03a32914e882ed315577005c472665e542419f910bab445815ad1929a7958f)


  0%|          | 0/2 [00:00<?, ?it/s]

In [10]:
ds_train = Dataset.from_pandas(df_coqa_wot)
ds_val = Dataset.from_pandas(df_1_span[(df_1_span['data_split']=='test')|(df_1_span['data_split']=='validation')].reset_index())
ds_test = Dataset.from_pandas(df_1_span[df_1_span['data_split']=='train'].reset_index())

ds_sq = DatasetDict({'train': ds_train,
                     'test': ds_test,
                     'validation': ds_val})

In [11]:
prefix = "question: "
max_input_length = 512
max_target_length = 50

def clean_text(text):
    question = nltk.sent_tokenize(text.split('SPLIT')[0])
    sentences = nltk.sent_tokenize(text.split('SPLIT')[1])
    text_cleaned = "\n ".join([" ".join(question).lower(), " ".join(sentences).lower()])
    return text_cleaned

def preprocess_data(examples):
    texts_cleaned = [clean_text(text) for text in examples["input"]]
    inputs = [prefix + text for text in texts_cleaned]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["output"], max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

def preprocess_test(examples):
    texts_cleaned = [clean_text(text) for text in examples["input"]]
    inputs = [prefix + text for text in texts_cleaned]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True,
                            padding="max_length")
    return model_inputs

In [12]:
tokenized_train = ds_sq['train'].map(preprocess_data, batched=True)
tokenized_dev = ds_sq['validation'].map(preprocess_data, batched=True)
tokenized_test = ds_sq['test'].map(preprocess_test, batched=True)

Map:   0%|          | 0/96987 [00:00<?, ? examples/s]

/home/ubuntu/QuestionAnswering/project-env/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:3581: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [39]:
per_device_eval_batch_size = 64     #64
per_device_train_batch_size = 8     #16
gradient_accumulation_steps = 8  

data_collator = DataCollatorForSeq2Seq(tokenizer)

args = Seq2SeqTrainingArguments(
    model_dir,
    evaluation_strategy="steps",
    eval_steps=100,
    logging_strategy="steps",
    logging_steps=10,
    save_strategy="steps",
    save_steps=100,
    learning_rate=4e-5,
    per_device_train_batch_size=per_device_train_batch_size,
    per_device_eval_batch_size=per_device_eval_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    weight_decay=0.01,
    predict_with_generate=True,
    fp16=True,
    load_best_model_at_end=True,
    metric_for_best_model="rouge1",
    report_to="tensorboard",
    optim="adafactor",
    max_steps=5000
)

model_checkpoint = model_name

def model_init():
    return AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint).to(device)

trainer = Seq2SeqTrainer(
    model_init=model_init,
    args=args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_dev,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()

PyTorch: setting up devices
loading configuration file config.json from cache at /home/ubuntu/.cache/huggingface/hub/models--allenai--unifiedqa-t5-large/snapshots/e59a9bfb34da29a50f83d15c9b570933f3044f21/config.json
Model config T5Config {
  "_name_or_path": "allenai/unifiedqa-t5-large",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "d_ff": 4096,
  "d_kv": 64,
  "d_model": 1024,
  "decoder_start_token_id": 0,
  "dense_act_fn": "relu",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": false,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 24,
  "num_heads": 16,
  "num_layers": 24,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty": 2.0,
      "ma

OutOfMemoryError: CUDA out of memory. Tried to allocate 256.00 MiB (GPU 0; 22.06 GiB total capacity; 20.69 GiB already allocated; 10.38 MiB free; 20.96 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [16]:
checkpoint = "checkpoint-700"
#model_dir = f"{model_dir}/{checkpoint}"
model_dir = '/home/ubuntu/QuestionAnswering/Notebooks/Models/extractive_WoT/allenai/unifiedqa-t5-base/checkpoint-700'
tokenizer = AutoTokenizer.from_pretrained(model_dir)
model = AutoModelForSeq2SeqLM.from_pretrained(model_dir)#.to(device)

max_input_length = 512

# prepare dataloader
tokenized_test.set_format(type='torch', columns=['input_ids', 'attention_mask'])
dataloader = torch.utils.data.DataLoader(tokenized_test, batch_size=32)

# generate text for each batch
all_predictions = []
for i,batch in enumerate(dataloader):
  predictions = model.generate(**batch)
  all_predictions.append(predictions)

# flatten predictions
all_predictions_flattened = [pred for preds in all_predictions for pred in preds]

# tokenize and pad titles
all_titles = tokenizer(tokenized_test["output"], max_length=max_target_length, truncation=True, padding="max_length")["input_ids"]

# compute metrics
predictions_labels = [all_predictions_flattened, all_titles]
compute_metrics(predictions_labels)


loading file spiece.model
loading file tokenizer.json
loading file added_tokens.json
loading file special_tokens_map.json
loading file tokenizer_config.json
loading configuration file /home/ubuntu/QuestionAnswering/Notebooks/Models/extractive_WoT/allenai/unifiedqa-t5-base/checkpoint-700/config.json
Model config T5Config {
  "_name_or_path": "/home/ubuntu/QuestionAnswering/Notebooks/Models/extractive_WoT/allenai/unifiedqa-t5-base/checkpoint-700",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "d_ff": 3072,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dense_act_fn": "relu",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": false,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 12,
  "num_heads": 12,
  "num_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,

{'rouge1': 32.145,
 'rouge2': 23.5364,
 'rougeL': 31.4358,
 'rougeLsum': 31.5187,
 'exact_match': 0.0711,
 'gen_len': 10.8472}

In [ ]:
# tokenize and pad titles
all_titles = tokenizer(tokenized_test["output"], max_length=max_target_length,
                       truncation=True, padding="max_length")["input_ids"]

# compute metrics
predictions_labels = [all_predictions_flattened, all_titles]
compute_metrics(predictions_labels)

{'rouge1': 25.3522,
 'rouge2': 16.3521,
 'rougeL': 24.872,
 'rougeLsum': 24.7843,
 'exact_match': 0.056,
 'gen_len': 7.7065}